# Multi-class Classification with Logistic Regression
This notebook makes use of logistic regression to come up with a multi-class classification model that tries to recognize handwritten digits. This is part one of exercise 3 in Prof. Ng's class and I'm closely following the [Python implementation](http://www.johnwittenauer.net/machine-learning-exercises-in-python-part-4/) provided by John Wittenhauer.

In [1]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.optimize import minimize

## Loading the Data
The data file is in a special Matlab matrix format, so we use a scipy function for handling this sort of file. 

In [2]:
data = loadmat('Data/ex04data1.mat')
type(data)

dict

In [3]:
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

The `data` variable points to a dict whose values are the `X` and `y` nd-arrays.

In [4]:
data['X'].shape

(5000, 400)

According to the exercise text, the `X` nd-array can be understood as follows:
> There are 5000 training examples in [ex04data1.mat], where each training example is a 20 pixel by 20 pixel grayscale image of the digit. Each pixel is represented by a floating point number indicating the grayscale intensity at that location. The 20 by 20 grid of pixels is "unrolled" into a 400-dimensional vector. Each of these training examples becomes a single row in our data matrix X. This gives us a 5000 by 400 matrix X where every row is a training example for a handwritten digit image.

In [5]:
data['y'].shape

(5000, 1)

The `y` nd-array contains the training labels for our training sample. According to the exercise text, the label of the digit '0' is actually set to 10 in order to avoid confusion with indexing since Matlab uses 1-indexing instead of 0-indexing.
So, to check whether the `y` dn-array contains the labels we expect:

In [6]:
labels = np.unique(data['y'])
labels

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [7]:
nclasses = len(labels)
nclasses

10

## The Sigmoid, Cost, and Gradient Functions
The sigmoid, cost, and gradient functions from the previous exercise are used here.

### The Sigmoid Function

In [8]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

### The Regularized Cost Function

In [9]:
def cost(theta, X, y, regrate=1):
    theta = np.matrix(theta).T
    X = np.matrix(X)
    y = np.matrix(y)
    m, _ = y.shape
    z = X * theta
    cost_i = np.log(sigmoid(z)) + np.diag(y.A1 - 1) * z
    reg_term = (regrate * np.sum(np.power(theta[1:], 2))) / (2 * m)
    return ((-np.sum(cost_i)) / m) + reg_term

### Regularized Gradient Function

In [10]:
def gradient(theta, X, y, regrate=1):
    theta = np.matrix(theta).T
    X = np.matrix(X)
    y = np.matrix(y)
    m = y.shape[0]
    n = theta.shape[0]
    
    z = X * theta
    
    reg_vec = np.matrix(np.zeros(shape=(n, 1)))
    reg_vec[1:] = regrate
    reg_diag = np.eye(n) * reg_vec
    
    grad = (X.T * (sigmoid(z) - y)) + reg_diag
    return (grad.A1) / m

## One-vs-all Classification
Since our objective is to classify handwritten digits from '0' to '9', we need to estimate 10 different classifiers (one for each digit).

The below function should return all the classifier parameters in a matrix `theta_all` where each row of `theta_all` corresponds to the learned logistic regression parameters for one class. (The below approach is very similar to the one implemented by John Wittenhauer in [part 4 of his ML blog series](http://www.johnwittenauer.net/machine-learning-exercises-in-python-part-4/).)

In [11]:
def one_vs_all(X, y, nclasses, regrate=1):
    nrows, nparams = X.shape
    
    # This is the k x (n + 1) matrix of all of the estimated coeffs
    # where k is the number of classes (labels)
    # n is the number of params
    # n + 1 is used to include the intercecpt param
    theta_all = np.zeros((nclasses, nparams + 1))
    
    # inserting a ones column for the intercept term
    X = np.insert(X, 0, values=np.ones(nrows), axis=1)
    
    # estimating each of the nclasses classifiers
    for i in range(nclasses):
        # '0' is represented as 10 in the data set
        digit = i + 1
        
        # This creates an nrows x 1 array which contains
        # 1 in the mth element if the label in mth elem of y is equal to `digit`
        # 0 otherwise
        y_digit = y == digit
        y_digit = y_digit.astype(int)   # convert bool to int
        
        # minimize the objective func
        fmin = minimize(fun=cost, x0=theta_all[i],
                        args=(X, y_digit, regrate), method='TNC',
                        jac=gradient)
        
        theta_all[i] = fmin.x
    
    return theta_all

## Computing the Optimal Thetas

In [12]:
theta_all = one_vs_all(X=data['X'], y=data['y'], nclasses=nclasses)

In [13]:
theta_all.shape

(10, 401)

In [14]:
theta_all

array([[-0.62490239, -0.21277825, -0.21277825, ..., -0.21250584,
        -0.21277796, -0.21277825],
       [-1.80262264, -0.38890201, -0.38890201, ..., -0.38676592,
        -0.38914839, -0.38890201],
       [-3.9025954 , -0.34165007, -0.34165007, ..., -0.34166276,
        -0.34164997, -0.34165007],
       ..., 
       [-7.10545462, -0.33453415, -0.33453415, ..., -0.33460472,
        -0.3345285 , -0.33453415],
       [-3.55434963, -0.31415108, -0.31415108, ..., -0.31450722,
        -0.314125  , -0.31415108],
       [-2.71456744, -0.55210593, -0.55210593, ..., -0.55147119,
        -0.55208727, -0.55210593]])

## Identifying Handwritten Digits
We now use the estimated values for our parameters in our set of logistic classifiers to identify a given handwritten digit. To do this we apply each of the 10 classifiers we estimated on each of the training sample and choose the classifier which gives  the highest sigmoid as the correct one for that particular example. This is implemented in the following function:

In [15]:
def identify_digit(X, theta_all):
    nrows, _ = X.shape
    
    X = np.insert(X, 0, values=np.ones(nrows), axis=1)
    
    X = np.matrix(X)
    theta_all = np.matrix(theta_all)
    
    # sigmoid for each class on a given training example
    # Each row of h contains the 10 sigmoids for that example
    h = sigmoid(X * theta_all.T)
    
    max_h_index = np.argmax(h, axis=1)
    
    return max_h_index + 1

In [17]:
y_hat = identify_digit(data['X'], theta_all)

In [18]:
y_hat.shape

(5000, 1)

In [19]:
y_hat[:5]

matrix([[10],
        [10],
        [10],
        [10],
        [10]])

In [20]:
y_hat[-5:]

matrix([[9],
        [9],
        [9],
        [9],
        [7]])

## Checking Accuracy

In [22]:
m, _ = y_hat.shape
correct = data['y'] == y_hat
accuracy = (np.sum(correct.astype(float)) / m) * 100
accuracy

93.679999999999993

Well, not bad for a first try, I guess. Prof. Ng's and Mr. Wittenhauer's accuracy values for this exercise are 95% and 98%, respectively.